In [1]:
import numpy as np
from copy import copy
import tensorflow as tf
from importlib import reload
from random import randint, sample, shuffle
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
size = plt.rcParams["figure.figsize"]
size[0] = 2
size[1] = 2
plt.rcParams["figure.figsize"] = size

import environment as env
import visual
import trainer as train
import agents.random_plus as rp
import agents.Q as Q
import agents.human as human

In [2]:
_ = reload(env)
_ = reload(train)
_ = reload(Q)
_ = reload(visual)
_ = reload(rp)
_ = reload(human)

In [48]:
# Create a trainable Q-Learning agent for a 3x3 game
QAgent = Q.Q([9, 100, 100, 100, 9], gamma = .6, epsilon = .1, beta = 5.0, name = "PlaygroundTest")
# Trainer for the agent
trainer = train.Trainer(QAgent)
# Create a random playing agent
RandomAgent = rp.RandomAgentPlus()
# Create Human agents
HumanP1 = human.Human()
HumanP2 = human.Human()
# Create an environment with a 3x3 board
e = env.Env(3)

In [73]:
# Train against a random agent in GUI mode
gui = visual.GameWithConfidences(e, QAgent, RandomAgent, max_games = -1, trainer_a1 = trainer.get_online(.00000001))

In [65]:
# Train against a random agent in normal mode
e.play(QAgent, RandomAgent, 1000, trainer_a1 = trainer.get_online(.00000001))

In [72]:
# Train against previously set human moves
s, a, r = [HumanP1.states, HumanP1.actions, HumanP1.rewards]
trainer.offline(s, a, r, epochs = 10, learn_rate = .00000001, batch_size = 10)

Training ********** Done
